In [1]:
import pandas as pd
from langchain.llms import Ollama
import re

# Load CSV
df = pd.read_csv("statement_report.csv")
df = df.dropna(subset=["Transaction Details"])  # Drop empty rows

# LLM setup
llm = Ollama(model="blueprint-financial-assistant")

# Format prompt system + output template
system_instruction = """
You are a financial assistant.
Your task is to classify each bank transaction strictly into EXPENSE or INCOME and provide a short note.

Rules:
- If 'PURCHASE AT' or 'PAYMENT TO' → classify as EXPENSE, note: "Shop purchase".
- If 'FUNDS TRANSFERRED TO' → classify as EXPENSE, note: "Outgoing transfer".
- If 'PAYMENT FROM' or 'FUNDS RECEIVED FROM' → classify as INCOME, note: "Incoming transfer".
- If none of these keywords are found, return UNKNOWN, note: "Other".
- Do not hallucinate.
- Output format: [Category] | [Note]
"""

# Function to extract standard label
def classify_transaction(details):
    prompt = f"{system_instruction}\nTransaction: {details}"
    response = llm.invoke(prompt).strip()

    # Validate format
    if re.match(r"^(EXPENSE|INCOME|UNKNOWN)\s\|\s.+", response):
        category, note = response.split(" | ", 1)
        return category, note
    else:
        return "UNKNOWN", "Other"

# Process
results = []
for _, row in df.iterrows():
    date = row['Date']
    details = row['Transaction Details']
    money_in = row['Money In']
    money_out = row['Money Out']
    balance = row['Balance']

    # Get classification and note
    category, note = classify_transaction(details)

    # Choose correct amount format
    if category == "INCOME":
        amount = f"{money_in} USD" if pd.notna(money_in) else ""
    elif category == "EXPENSE":
        amount = f"{money_out} USD" if pd.notna(money_out) else ""
    else:
        amount = ""

    results.append({
        "Date": date,
        "Transaction Details": details,
        "Expense/Income": category,
        "USD": amount,
        "Notes": note,
        "Balance (USD)": balance
    })

# Convert to DataFrame
final_df = pd.DataFrame(results)

# (Optional) Normalize category labels
def normalize_category(note):
    if note == "Other":
        return "Other"
    if note == "Shop purchase":
        return "Shopping"
    if note in ["Incoming transfer", "Outgoing transfer"]:
        return "Transfers"
    return "Other"

final_df["Normalized Category"] = final_df["Notes"].apply(normalize_category)

# Export
final_df.to_csv("standardized_classified_transactions.csv", index=False)
print(final_df.head())

/tmp/ipykernel_30353/484668021.py:10: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="blueprint-financial-assistant")


       Date                                Transaction Details Expense/Income  \
0  1-Oct-24  PAYMENT FROM LY PHENGHOR 016847399 BANK ACLEDA...         INCOME   
1  1-Oct-24  PURCHASE AT LENG THOEURNG ON Oct 01, 2024 07:1...        EXPENSE   
2  1-Oct-24  FUNDS RECEIVED FROM PHENG ROTHA (004 179 676) ...         INCOME   
3  1-Oct-24  FUNDS RECEIVED FROM MET TONG (003 667 540) ORI...         INCOME   
4  5-Oct-24  PAYMENT TO Metfone (PIN-less) 0312751111 ORIGI...        EXPENSE   

       USD                Notes  Balance (USD) Normalized Category  
0  8.0 USD  "Incoming transfer"           8.48               Other  
1  8.0 USD        Shop purchase           0.48            Shopping  
2  8.0 USD  "Incoming transfer"           8.48               Other  
3  8.0 USD  "Incoming transfer"          16.48               Other  
4  1.0 USD        Shop purchase          15.48            Shopping  
